In [2]:
!pip install transformers[torch] --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.6 MB/s eta 0:00:00
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.2 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.0/785.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.8 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 9.5 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 16.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 54.7 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/2

In [23]:
PATH = '/home/jimi/unicamp/bart_roberta/'
dados_treino = '/home/jimi/unicamp/bart_roberta/texto.txt'

In [24]:
from tokenizers import ByteLevelBPETokenizer
from transformers import RobertaTokenizer
from transformers import RobertaConfig

In [9]:
# Initialize a ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=[dados_treino], vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"
    ])

In [10]:
tokenizer.encode("BitDogLab").ids

[38, 318, 380]

In [16]:
#neste momento, nosso modelo já possui um tokenizer construído a partir dos dados
#vocab.json, lista dos tokens ordenados por frequência - converte tokens para IDs 
#merges.txt - mapeia textos para tokens

!rm -r /home/jimi/unicamp/bart_roberta/RAW_MODEL
!mkdir /home/jimi/unicamp/bart_roberta/RAW_MODEL


rm: não foi possível remover '/home/jimi/unicamp/bart_roberta/RAW_MODEL': Arquivo ou diretório inexistente


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
tokenizer.save_model(PATH+'RAW_MODEL')

['/home/jimi/unicamp/bart_roberta/RAW_MODEL/vocab.json',
 '/home/jimi/unicamp/bart_roberta/RAW_MODEL/merges.txt']

In [25]:
from transformers import RobertaConfig

In [27]:
tokenizer = RobertaTokenizer.from_pretrained(PATH+'RAW_MODEL', max_len=512)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [29]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [30]:
model.num_parameters()

83502880

In [31]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=dados_treino,
    block_size=128,
)

/home/jimi/.local/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [32]:
# verificando
dataset.examples[:2]

[{'input_ids': tensor([  0,  51,  70,  78,  73, 602, 313, 410, 381,  16, 373, 339,  77, 425,
          531, 313, 727, 394, 530, 264, 297, 517,  18,  20, 299, 370, 511, 510,
           16, 356, 373, 596, 802, 771, 274,  72, 711, 681, 259, 618, 263, 698,
           89, 272, 364,  18, 316, 302,  73, 427, 363, 392, 446, 486, 569, 691,
          678,  16, 612, 475, 499, 776,  16, 539,  86, 263, 772, 703, 354, 447,
          613, 539, 441, 363, 759, 348, 263, 494, 390, 777, 597, 274, 529, 270,
          663, 350, 601, 263, 408,  75, 465,  18, 386, 367, 289,  77, 707,  87,
          713, 298,  71, 432, 586, 411,  16, 359, 613, 312,  81, 512, 505, 362,
          493, 311, 284, 363, 769,  87,  69,  16, 808, 359, 499, 265, 750, 572,
          334,   2])},
 {'input_ids': tensor([  0,  42, 397, 771, 527, 299, 348, 381,  30, 741, 684, 268, 712, 454,
          418, 271,  71, 463, 739,  87, 299, 381, 263, 474, 608, 261, 355, 275,
          425, 309, 796, 523, 419, 671, 331,  87, 519, 748,  84, 581, 3

In [34]:
'''
Data Collators são estratégias de se construir lotes de dados 
para treinar o modelo. Cria listas de amostras a partir do dataset e 
permite que o Pytorch aplique a backpropagation adequadamente.
Probability = probabilidade de mascarar tokens da entrada
'''
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.1
)

In [35]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=PATH+'RAW_MODEL',
    overwrite_output_dir=True,
    num_train_epochs=1200,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [36]:
trainer.train()

Step,Training Loss
500,2.751000
1000,0.032800


TrainOutput(global_step=1200, training_loss=1.162301374077797, metrics={'train_runtime': 130.1388, 'train_samples_per_second': 27.663, 'train_steps_per_second': 9.221, 'total_flos': 119364422860800.0, 'train_loss': 1.162301374077797, 'epoch': 1200.0})

In [37]:
trainer.save_model(PATH+'RAW_MODEL')

In [41]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=PATH+'RAW_MODEL',
    tokenizer=PATH+'RAW_MODEL'
)

In [48]:
texto = 'a BitDogLab é uma <mask>'

In [49]:
fill_mask(texto)

[{'score': 0.7000885605812073,
  'token': 299,
  'token_str': ' da',
  'sequence': 'a BitDogLab é uma da'},
 {'score': 0.2806237041950226,
  'token': 602,
  'token_str': 'tivo',
  'sequence': 'a BitDogLab é umativo'},
 {'score': 0.0007573068724013865,
  'token': 263,
  'token_str': ' e',
  'sequence': 'a BitDogLab é uma e'},
 {'score': 0.0006443043821491301,
  'token': 274,
  'token_str': ' de',
  'sequence': 'a BitDogLab é uma de'},
 {'score': 0.00025438613374717534,
  'token': 87,
  'token_str': 's',
  'sequence': 'a BitDogLab é umas'}]